# Dependency Parsing of Bengali Language

In [1]:
import os , sys ,re , json , time , glob
from tqdm import tqdm 
import copy
import os
import sys,re
import numpy as np
from scipy import sparse
from collections import Counter,defaultdict
from tqdm.notebook import tqdm

## Preprocessing 

In [2]:
sent_id = 0
# final_list = list()
chunks_with_troot = list()
error_list = set()

In [3]:
def clean(txt):
    temp = re.sub(r'\s+', '', txt)
    temp = re.sub(r'[<\'">]', '', temp)
    # temp = re.sub(r'"', '', temp)
    return temp


def parse_normal_line(line, file_name):
    token = line.split('\t')
    word = token[1]
    word_pos_tag = token[2]
    return word , word_pos_tag


def parse_1st_line(line1, file_name):
    line = re.sub('\t', ' ' , line1)
    tokens = line.split(' ')
    name_found = False
    af, pos_name , true_pos_name ,  dep_rel , head , leader , word = '', '', '', '', '', '', ''
    for tkn in tokens:
        if 'af=' in tkn:
            af = tkn
        elif 'name=' in tkn :
            name_found = True
            pos_name = clean(tkn.split('=')[1])
            if len(pos_name) >= 2 and pos_name[-2].isdigit(): true_pos_name = pos_name[:-2]
            elif len(pos_name) >= 1 and pos_name[-1].isdigit() : true_pos_name = pos_name[:-1]
            else: true_pos_name = pos_name
        elif 'rel=' in tkn :
            if "rel=''" in tkn:
                continue
            item = tkn.split('=')[1]
            try :
                dep_rel , head = item.split(':')
            except :
                print(file_name , line)
            dep_rel , head = clean(dep_rel) , clean(head)
        elif 'head=' in tkn:
            leader = clean(tkn.split('=')[1])
        elif 'troot=' in tkn:
            word = clean(tkn.split('=')[1])


    if name_found == False:
        pos_name = tokens[2]
    return  af, pos_name , true_pos_name ,  dep_rel , head , leader , word 


def parse_root_chunk(chunk , id, file_name):
    first_line = chunk[0]
    af, pos_name , true_pos_name ,  dep_rel , head , leader , word  = parse_1st_line(first_line, file_name)
    
    chunk_list =[]
    rest_lines = chunk[1:]
    for one_line in rest_lines:
        word2 , word_pos_tag = parse_normal_line(one_line, file_name)
        if word2 == 'NULL':
            local_word = word
            if leader == 'NULL' :
                leader = word
        else : 
            local_word = word2
        chunk_list.append([id , 
                          local_word , 
                          word_pos_tag ,
                          af, 
                          pos_name , 
                          true_pos_name ,  
                          dep_rel , 
                          head , 
                          leader ])
    return chunk_list
  

def parse_normal_chunk(chunk , id, file_name):
    first_line = chunk[0]
    af, pos_name , true_pos_name ,  dep_rel , head , leader , word  = parse_1st_line(first_line, file_name)

    chunk_list = []
    rest_lines = chunk[1:]
    for one_line in rest_lines:
        word2 , word_pos_tag = parse_normal_line(one_line, file_name)
        if word2 == 'NULL':
            local_word = word
            if leader == 'NULL' :
                leader = word
        else : 
            local_word = word2
        chunk_list.append([id , 
                          local_word , 
                          word_pos_tag ,
                          af, 
                          pos_name , 
                          true_pos_name ,  
                          dep_rel , 
                          head , 
                          leader] )
        
    return chunk_list


def troot_chunk(chunk):
    global chunks_with_troot
    if 'troot=' in chunk[0]:
        chunks_with_troot.append(chunk)


def chunk_parser(chunk, id, file_name) :
    is_root = False
    first_line = chunk[0]

    if 'rel=' in first_line:
        if "rel=''" in first_line :
            chunk_list = parse_root_chunk(chunk , id, file_name)
        else:
            chunk_list = parse_normal_chunk(chunk , id, file_name)
    else:
        chunk_list = parse_root_chunk(chunk , id, file_name)
        is_root = True
    return chunk_list


def sentence_parser(line):
    tokens = line.split(' ')
    variables = ''
    for tkn in tokens:
        if 'name' in tkn:
            big_pos = clean(tkn.split('=')[1])
        elif 'troot=' in tkn:
            word = clean(tkn.split('=')[1])
        elif 'drel=' in tkn or 'dmrel=' in tkn:
            item = tkn.split('=')[1]
            dep_rel , root = item.split(':')
            dep_rel , root = clean(dep_rel) , clean(root)



def document_parser(file, file_name, final_list):
    # read file... usimg readlines such that len(file) = number of lines...
    global sent_id
    # global final_list
    for idx in range(len(file)):
        if '<Sentence' in file[idx]:
            sent_id += 1
        elif '</Sentence' in file[idx]:
            continue
        elif '((' in file[idx]:
            chunk = []
            while '))' not in file[idx]:
                chunk.append(file[idx])
                idx += 1
            # troot_chunk(chunk)
            chunk_list = chunk_parser(chunk , sent_id, file_name)
            final_list.append(chunk_list)
            


In [4]:
def prepare_data1(folder_path) :
    global sent_id, chunks_with_troot, error_list
    final_list = list()
    for dirpath, dirnames, files in os.walk(folder_path):
        for file_name in files:
            path = os.path.join(folder_path,file_name)
            with open(path, "r") as fp :
                lines = fp.read().splitlines()
                document_parser(lines, file_name, final_list)

    needed_list = list()
    for lists in final_list:
        for l in lists:
            needed_list.append(l)
    return needed_list        

In [5]:
cc = 0        

def find_head_pos(chunk , head):
    global cc , error_list
    idx = 1

    if head == '':
        for word_line in chunk:
            if head == word_line[7] and word_line[1] == word_line[8]   : #pos_name and local_word == leader i.e word_line[1] == word_line[8]  
                return idx
            idx += 1
        
    for word_line in chunk:
        if head == word_line[4] and word_line[1] == word_line[8]   : #pos_name and local_word == leader i.e word_line[1] == word_line[8]  
            return idx
        idx += 1
    cc += 1

    error_list.add(chunk[0][0])
    return -9999
    

def process_sentence(recv_chunk):
    chunk = copy.deepcopy(recv_chunk)
    for word_line in chunk:
        word_line = word_line.append(-1)


    for word_line in chunk:
        if word_line[6] == '' and word_line[7] =='' and word_line[1] == word_line[8]  : #head and dep_rel field '' also  local_word == leader
            word_line[9] = 0
            word_line[6] = 'root'
    len1 = len(chunk)
    
    for word_line in chunk:
        head = word_line[7]
        if word_line[9] == 0 : continue
        head_pos = find_head_pos(chunk , head)
        word_line[9] = head_pos
    return chunk


In [6]:
def prepare_data2(req_list) :
    global error_list
    new_idx = req_list[0][0]
    temp_chunk = list()
    ultimate_list = list()
    
    for line in req_list:
        if line[0] == new_idx:
            temp_chunk.append(line)
        else:
            ans = process_sentence(temp_chunk)
            for a in ans :
                ultimate_list.append(a)
            new_idx = line[0]
            temp_chunk = list()
            temp_chunk.append(line)

    ans = process_sentence(temp_chunk)
    for a in ans :
        ultimate_list.append(a)
    temp_list = copy.deepcopy(ultimate_list)
    ccc = 0
    for i in temp_list :
        if i[0] in error_list:
            ccc += 1
            ultimate_list.remove(i)
    return ultimate_list

### The information that are extracted from each sentence

In [7]:
# New structure :

# id ,                  0
# local_word ,          1
# word_pos_tag ,        2
# af,                   3
# pos_name ,            4
# true_pos_name ,       5
# dep_rel ,             6
# head ,                7
# leader                8



# head_line             9

### path for the dataset 

In [8]:
train_dir = '/content/train'  #folder with all the head files
test_dir = '/content/test'

In [50]:
temp_train = prepare_data1(train_dir)
train_treebank = prepare_data2(temp_train)

### Total number of sentences in training dataset

In [10]:
len(train_treebank)

87014

### Sample example from the training treebank after pre-processing

In [11]:
train_treebank[10:13]

[[1,
  'ভরে',
  'V__VM__VNF',
  "af='None'",
  'VGNF',
  'VGNF',
  'vmod',
  'VGNN2',
  'ভরে',
  15],
 [1,
  'আমাজনের',
  'N__NNP',
  "af='None'",
  'NP5',
  'NP',
  'r6',
  'NP6',
  'আমাজনের',
  13],
 [1,
  'জলজঙ্গলে',
  'N__NN',
  "af='None'",
  'NP6',
  'NP',
  'k7p',
  'VGNN2',
  'জলজঙ্গলে',
  15]]

In [12]:
temp_test = prepare_data1(test_dir)
test_treebank = prepare_data2(temp_test)

### Total number of sentences in testing dataset

In [13]:
len(test_treebank)

11467

### Sample example from the testing treebank after pre-processing

In [14]:
test_treebank[10:13]

[[7205,
  'মার্টিনা',
  'N__NNP',
  "af='None'",
  'NP6',
  'NP',
  'k1',
  'NULL__VGF',
  'হিঙ্গিস',
  17],
 [7205,
  'হিঙ্গিস',
  'N__NNP',
  "af='None'",
  'NP6',
  'NP',
  'k1',
  'NULL__VGF',
  'হিঙ্গিস',
  17],
 [7205,
  'অস্ট্রেলিয়ান',
  'JJ',
  "af='None'",
  'NP7',
  'NP',
  'r6',
  'NP8',
  'ওপেনের',
  15]]

# Dependency Parser

In [15]:
# global data
all_features = list()
all_labels = list()

## Check if a Tree is Connected or not

In [16]:
def check_if_tree_is_connected(begin, dependent_head, token_heads ,line_tokens) :     
    current = begin
    while current is not 0:
        current = token_heads[current]
        if current == dependent_head:
            return True
    return False

## Check Provectivity

In [17]:
def check_projectivity(line_tokens) :
    token_heads = defaultdict()
    for fields in line_tokens:
        token_id = fields[0]
        token_head = fields[3]
        
        if token_heads.get(token_id) == None: # not seen before
            token_heads[token_id] = token_head

    for dependent in token_heads:
        dependent_head = token_heads[dependent]
        left = 1 + min(dependent_head,dependent)
        right = max(dependent_head,dependent)
        
        for i in range(left,right):
            if check_if_tree_is_connected(i, dependent_head, token_heads ,line_tokens) == False:
                return False
    return True

## Checking conditions for selecting Arc

In [18]:
def check_condition(buffer,stack,dependencies = {},condition=None):    
    if(condition == '0'):
        return len(buffer) is 0 and len(stack) is 1 and stack[0] == 0
    
    elif(condition == '1'):
        return len(stack) < 2 and len(buffer) > 0 #stack has only root and items are still in buffer

    elif(condition == '2'): #condition for left arc
        stack_top = stack[-1]
        stack_2nd = stack[-2]
        
        return dependencies.get(stack_top) is not None and (stack_top,stack_2nd) in dependencies[stack_top]
    
    elif(condition == '3'): #condition for right arc
        stack_top = stack[-1]
        stack_2nd = stack[-2]
        
        return dependencies.get(stack_2nd) is not None and (stack_2nd,stack_top) in dependencies[stack_2nd]
        

### Extracting Configuration States and Gold Transitions

In [19]:
def tree_to_actions(buffer,stack,arcs,dependencies):
    def perform_shift(buffer,stack,arcs,configuration_states,gold_transitions):
            configuration_states.append((list(buffer), list(stack), list(arcs)))

            gold_transitions.append("shift")

            stack.append(buffer.pop())
    
    def perform_arc(arc_label,buffer,stack,arcs,configuration_states,gold_transitions,arc_type):
        configuration_states.append((list(buffer), list(stack), list(arcs)))
        
        action = f'{arc_type}_{arc_label}'
        
        gold_transitions.append(action)
        
        left = stack[-1]
        right = stack[-2]
        
        if arc_type == 'left':
            arcs.append((arc_label,left,right))
            stack.pop(-2)
            
        elif arc_type == 'right':
            arcs.append((arc_label,right,left))
            stack.pop()
    
    configuration_states = list() #[(buf,stk,arc)...]
    gold_transitions = list() #[shift or left_rel or right_rel] actions for each state
    
    seen_dependencies = defaultdict()
    
    while( len(buffer) >= 0 ):
        if check_condition(buffer,stack,condition = '0'):
            return configuration_states , gold_transitions
        
        if check_condition(buffer,stack,condition = '1'):
            perform_shift(buffer,stack,arcs,configuration_states,gold_transitions)
            continue
            
        stack_top = stack[-1]
        stack_2nd = stack[-2]
        
        if check_condition(buffer,stack,dependencies,condition = '2'):
            arc_label = dependencies[stack_top][(stack_top,stack_2nd)]
            
            perform_arc(arc_label, buffer,stack,arcs,
                        configuration_states,gold_transitions,
                        arc_type = 'left')
            
            seen_dependencies[stack_2nd] = 1
            
        elif check_condition(buffer,stack,dependencies,condition = '3'):
            if ( dependencies.get(stack_top) is not None and any([child not in seen_dependencies for _,child in dependencies[stack_top]]) ):
                
                perform_shift(buffer,stack,arcs,configuration_states,gold_transitions)
                
            else:                 
                arc_label = dependencies[stack_2nd][(stack_2nd,stack_top)]
            
                perform_arc(arc_label, buffer,stack,arcs,
                        configuration_states,gold_transitions,
                        arc_type = 'right')
            
                seen_dependencies[stack_top] = 1

        else:
            if len(buffer) > 0 :
                perform_shift(buffer,stack,arcs,configuration_states,gold_transitions)
            else :
                return configuration_states , gold_transitions

In [20]:
def extract_configs_transitions(line_tokens):    
    stack,buffer,arcs = list(),list(),list()
    dependencies = defaultdict(lambda: defaultdict())

    stack.append(0) #add root
    
    for tokens in reversed(line_tokens):
        tok_id,word,x_pos,head,dependecy_relation = tokens
        head_tok_id = (head,tok_id)
        dependencies[head][head_tok_id] = dependecy_relation
        buffer.append(tok_id)
        
    return tree_to_actions(buffer,stack,arcs,dependencies)
   

## Extract Features

In [21]:
def countLeftRight(labels) :
    vl, vr = 0, 0
    for label in labels :
        if label.startswith('left') : vl += 1
        elif label.startswith('right') : vr += 1
    return vl, vr   

### Base line features

In [ ]:
def get_features_labels(current_configuration,current_transtion,tokens_dict,POS_tags):
    def get_stack_features(idx,stack,tokens_dict,POS_tags):
        if idx == 'top':
            return tokens_dict[stack[-1]],POS_tags[stack[-1]]
        elif idx == '2nd_top':
            return tokens_dict[stack[-2]],POS_tags[stack[-2]]
    
    def get_buffer_features(buffer,tokens_dict,POS_tags):
        return tokens_dict[buffer[-1]],POS_tags[buffer[-1]]    
        
    def get_two_word_features(tokens_dict,stack,POS_tags):
        return tokens_dict[stack[-1]],POS_tags[stack[-1]],POS_tags[stack[-2]]


    def insert_to_features(features,feat_i,feat_j):
        features[feat_i] = 1
        features[feat_j] = 1
        
        
    
    label = current_transtion
    
    buffer,stack,arcs = current_configuration
    
    features = defaultdict()
    
    if len(stack) > 0:
        top_token,top_pos = get_stack_features('top',stack,tokens_dict,POS_tags)
    
        top_feature_1 = f'stack_top_{top_token}'
        top_feature_2 = f'stack_pos_{top_pos}'
        
        insert_to_features(features,top_feature_1,top_feature_2)
    
    if len(stack) > 1:
        top2nd_token,top2nd_pos = get_stack_features('2nd_top',stack,tokens_dict,POS_tags)
    
        top2nd_feature_4 = f'stack2nd_top_{top2nd_token}'
        top2nd_feature_5 = f'stack2nd_pos_{top2nd_pos}'
        
        insert_to_features(features,top2nd_feature_4,top2nd_feature_5)
        
    if len(buffer) > 0:
        buf_token,buf_pos = get_buffer_features(buffer,tokens_dict,POS_tags)
    
        buf_feature_7 = f'buffer_top_{buf_token}'
        buf_feature_8 = f'buffer_pos_{buf_pos}'

        insert_to_features(features,buf_feature_7,buf_feature_8)
        
    
    return features,label    

### Baseline plus features

In [22]:
 def get_features_labels(current_configuration,current_transtion,tokens_dict,POS_tags):
    def get_stack_features(idx,stack,tokens_dict,POS_tags):
        if idx == 'top':
            return tokens_dict[stack[-1]],POS_tags[stack[-1]]
        elif idx == '2nd_top':
            return tokens_dict[stack[-2]],POS_tags[stack[-2]]
    
    def get_buffer_features(buffer,tokens_dict,POS_tags):
        return tokens_dict[buffer[-1]],POS_tags[buffer[-1]]    
        
    def get_two_word_features(tokens_dict,stack,POS_tags):
        return tokens_dict[stack[-1]],POS_tags[stack[-1]],POS_tags[stack[-2]]
    
    def insert_to_features(features,feat_i,feat_j,feat_k):
        features[feat_i] = 1
        features[feat_j] = 1
        features[feat_k] = 1        
        
    
    labels = current_transtion
    
    buffer,stack,arcs = current_configuration
    
    features = defaultdict()
    
    if len(stack) > 0:
        top_token,top_pos = get_stack_features('top',stack,tokens_dict,POS_tags)
    
        top_feature_1 = f'stack_top_{top_token}'
        top_feature_2 = f'stack_pos_{top_pos}'
        top_feature_3 = f'stack_top_pos_{top_token}_{top_pos}'
        
        insert_to_features(features,top_feature_1,top_feature_2,top_feature_3)
        # insert_to_features(features,top_feature_1,top_feature_2)
    
    if len(stack) > 1:
        top2nd_token,top2nd_pos = get_stack_features('2nd_top',stack,tokens_dict,POS_tags)
    
        top2nd_feature_4 = f'stack2nd_top_{top2nd_token}'
        top2nd_feature_5 = f'stack2nd_pos_{top2nd_pos}'
        top2nd_feature_6 = f'stack2nd_top_pos_{top2nd_token}_{top2nd_pos}'
        
        insert_to_features(features,top2nd_feature_4,top2nd_feature_5,top2nd_feature_6)
        
        
    if len(buffer) > 0:
        buf_token,buf_pos = get_buffer_features(buffer,tokens_dict,POS_tags)
    
        buf_feature_7 = f'buffer_top_{buf_token}'
        buf_feature_8 = f'buffer_pos_{buf_pos}'
        buf_feature_9 = f'buffer_top_pos_{buf_token}_{buf_pos}'
        
        insert_to_features(features,buf_feature_7,buf_feature_8,buf_feature_9)
        
        
    if len(stack) > 1 : 
        stack_top , pos_top , pos_2nd_top = get_two_word_features(tokens_dict,stack,POS_tags)
        
        two_word_feature_10 = f'stack_top_pos_top_pos_2ndtop_{stack_top}_{pos_top}_{pos_2nd_top}'
        features[two_word_feature_10] = 1

# baseline +

    if len(labels) > 0 :
        # valency
        vl, vr = countLeftRight(labels)
        if len(stack) > 0 and len(buffer) > 0 and ('shift' in labels[-1]) :
        # if len(stack) > 0 and len(buffer) > 0 :
            stack_top, stack_top_pos = tokens_dict[stack[-1]], POS_tags[stack[-1]]
            buffer_top, buffer_top_pos = tokens_dict[buffer[-1]], POS_tags[buffer[-1]]
            new_feature_1 = f'stack_top_valL_{stack_top}_{str(vl)}'
            new_feature_2 = f'stack_pos_valL_{stack_top_pos}_{str(vl)}'
            new_feature_3 = f'stack_top_valR_{stack_top}_{str(vr)}'
            new_feature_4 = f'stack_pos_valR_{stack_top_pos}_{str(vr)}'
            new_feature_5 = f'buffer_top_valL_{buffer_top}_{str(vl)}'
            new_feature_6 = f'buffer_pos_valL_{buffer_top_pos}_{str(vl)}'
            
            features[new_feature_1] = 1
            features[new_feature_2] = 1
            features[new_feature_3] = 1
            features[new_feature_4] = 1
            features[new_feature_5] = 1
            features[new_feature_6] = 1


        # distance
        if len(stack) > 0 and len(buffer) > 0 and ('left' in labels[-1] or 'right' in labels[-1]) : 
        # if len(stack) > 0 and len(buffer) > 0 : 
            stack_top, stack_top_pos = tokens_dict[stack[-1]], POS_tags[stack[-1]]
            buffer_top, buffer_top_pos = tokens_dict[buffer[-1]], POS_tags[buffer[-1]]    
            distance = abs(int(stack[-1]) - int(buffer[-1]))
            new_feature_1 = f'stack_top_dis_{stack_top}_{str(distance)}'
            new_feature_2 = f'stack_pos_dis_{stack_top_pos}_{str(distance)}'
            new_feature_3 = f'buffer_top_dis_{buffer_top}_{str(distance)}'
            new_feature_4 = f'buffer_pos_dis_{buffer_top_pos}_{str(distance)}'
            new_feature_5 = f'stack_top_buffer_top_dis_{stack_top}_{buffer_top}_{str(distance)}'
            new_feature_6 = f'stack_pos_buffer_pos_dis_{stack_top_pos}_{buffer_top_pos}_{str(distance)}'
            features[new_feature_1] = 1
            features[new_feature_2] = 1
            features[new_feature_3] = 1
            features[new_feature_4] = 1
            features[new_feature_5] = 1
            features[new_feature_6] = 1

    if len(labels) > 0 :
        return features,labels[-1]
    else :
        return features,labels

## Build Oracles

In [45]:
def build_oracles(train_treebank):
    global all_features, all_labels
    current_tokens = []
    tokens_dict = {}
    POS_tags = {}
    
    tokens_dict[0]= 'root'
    POS_tags[0]= 'root'

    new_idx = train_treebank[0][0]
    for current_line in tqdm(train_treebank):
        if current_line[0] == new_idx:
            tok_id,word,x_pos,head_id,dep_rel = current_line[0], current_line[1], current_line[2], current_line[-1], current_line[6] 
            current_tokens.append((tok_id,word,x_pos,head_id,dep_rel)) 
        else:
            new_current_tokens = []
            cnt_root = 0
            for idx , line in enumerate(current_tokens):
                temp = list(line)
                temp[0] = idx + 1
                new_current_tokens.append(tuple(temp))
                if temp[-1] == 'root' :
                    cnt_root += 1
                tokens_dict[temp[0]] = temp[1]
                POS_tags[temp[0]] = temp[2]
            
            if cnt_root == 1 and check_projectivity(new_current_tokens) == True: #if current tree is projective
                configuration_state,gold_transitions = extract_configs_transitions(new_current_tokens)
                print(configuration_state)
                print("\n\n")
                print(gold_transitions)
                print("\n\n")
                print(tokens_dict)
                
                for i in range(len(configuration_state)):
                    features , labels = get_features_labels(configuration_state[i],gold_transitions[:i+1], tokens_dict, POS_tags)

                    all_features.append(features)
                    all_labels.append(labels)
            
            current_tokens = []
            tokens_dict = {}
            POS_tags = {}
            
            tokens_dict[0]= 'root'
            POS_tags[0]= 'root'
            new_idx = current_line[0]
    
            tok_id,word,x_pos,head_id,dep_rel = current_line[0], current_line[1], current_line[2], current_line[-1], current_line[6] 
            current_tokens.append((tok_id,word,x_pos,head_id,dep_rel))

    new_current_tokens = []
    cnt_root = 0
    for idx , line in enumerate(current_tokens):
        temp = list(line)
        temp[0] = idx + 1
        new_current_tokens.append(tuple(temp))
        if temp[-1] == 'root' :
            cnt_root += 1
        tokens_dict[temp[0]] = temp[1]
        POS_tags[temp[0]] = temp[2]
    if cnt_root == 1 and check_projectivity(new_current_tokens) == True: #if current tree is projective
        configuration_state,gold_transitions = extract_configs_transitions(new_current_tokens)
        
        for i in range(len(configuration_state)):
            features , labels = get_features_labels(configuration_state[i],gold_transitions[:i+1], tokens_dict, POS_tags)

            all_features.append(features)
            all_labels.append(labels)

    print('finished')


In [24]:
build_oracles(train_treebank)


finished


### Sample output of Configuration state, Gold transitions, token dictionary

In [54]:
build_oracles(train_treebank[:100])

[([5, 4, 3, 2, 1], [0], []), ([5, 4, 3, 2], [0, 1], []), ([5, 4, 3], [0, 1, 2], []), ([5, 4, 3], [0, 2], [('k1', 2, 1)]), ([5, 4], [0, 2, 3], [('k1', 2, 1)]), ([5, 4], [0, 2], [('k1', 2, 1), ('k7p', 2, 3)]), ([5], [0, 2, 4], [('k1', 2, 1), ('k7p', 2, 3)]), ([5], [0, 2], [('k1', 2, 1), ('k7p', 2, 3), ('k7p', 2, 4)]), ([], [0, 2, 5], [('k1', 2, 1), ('k7p', 2, 3), ('k7p', 2, 4)]), ([], [0, 2], [('k1', 2, 1), ('k7p', 2, 3), ('k7p', 2, 4), ('rsym', 2, 5)])]



['shift', 'shift', 'left_k1', 'shift', 'right_k7p', 'shift', 'right_k7p', 'shift', 'right_rsym', 'right_root']



{0: 'root', 1: 'তিনি', 2: 'ছিলেন', 3: 'ব্রাজিল', 4: 'সেনাবাহিনীতে', 5: '।'}
[([6, 5, 4, 3, 2, 1], [0], []), ([6, 5, 4, 3, 2], [0, 1], []), ([6, 5, 4, 3], [0, 1, 2], []), ([6, 5, 4, 3], [0, 2], [('r6', 2, 1)]), ([6, 5, 4], [0, 2, 3], [('r6', 2, 1)]), ([6, 5], [0, 2, 3, 4], [('r6', 2, 1)]), ([6], [0, 2, 3, 4, 5], [('r6', 2, 1)]), ([6], [0, 2, 3, 5], [('r6', 2, 1), ('pof', 5, 4)]), ([6], [0, 2, 5], [('r6', 2, 1), ('pof', 5, 4

### Sample example of features

In [25]:
all_features[10:13]

[defaultdict(None,
             {'buffer_pos_N__NNP': 1,
              'buffer_pos_valL_N__NNP_0': 1,
              'buffer_top_pos_ব্রাজিলের_N__NNP': 1,
              'buffer_top_valL_ব্রাজিলের_0': 1,
              'buffer_top_ব্রাজিলের': 1,
              'stack_pos_root': 1,
              'stack_pos_valL_root_0': 1,
              'stack_pos_valR_root_0': 1,
              'stack_top_pos_root_root': 1,
              'stack_top_root': 1,
              'stack_top_valL_root_0': 1,
              'stack_top_valR_root_0': 1}),
 defaultdict(None,
             {'buffer_pos_N__NN': 1,
              'buffer_pos_valL_N__NN_0': 1,
              'buffer_top_pos_প্রকৃতি_N__NN': 1,
              'buffer_top_valL_প্রকৃতি_0': 1,
              'buffer_top_প্রকৃতি': 1,
              'stack2nd_pos_root': 1,
              'stack2nd_top_pos_root_root': 1,
              'stack2nd_top_root': 1,
              'stack_pos_N__NNP': 1,
              'stack_pos_valL_N__NNP_0': 1,
              'stack_pos_valR_N__NN

### Sample example of labels

In [26]:
all_labels[10:20]

['shift',
 'shift',
 'left_r6',
 'shift',
 'shift',
 'shift',
 'left_pof',
 'left_k2',
 'left_k1',
 'shift']

In [27]:
label_vocabulary = defaultdict()
feature_vocabulary = defaultdict()
reverse_features = list()
reverse_labels = list()
max_features = 0

In [28]:
def create_reverse_mappings(label_id, feature_id):
    global all_features, all_labels, label_vocabulary, feature_vocabulary, reverse_features, reverse_labels
        
    reverse_labels = [0] * label_id # empty array of number of labels
    reverse_features = [0] * feature_id # empty array of number of features
    
    for feature in tqdm(feature_vocabulary):
        reverse_features[feature_vocabulary[feature]] = feature
    for label in tqdm(label_vocabulary):
        reverse_labels[label_vocabulary[label]] = label


## Build Training Data 

In [29]:
def convert_to_training_data():
    global all_features, all_labels, label_vocabulary, feature_vocabulary, max_features
    label_id = 0
    feature_id = 0 
    
    feature_counts = defaultdict(int)
    total_features = len(all_features)
    
    for i in tqdm(range(total_features)):
        for feature in all_features[i]:
            feature_counts[feature] += 1
            
        if label_vocabulary.get(all_labels[i]) is None:
            label_vocabulary[all_labels[i]] = label_id
            label_id += 1
            
    for feature in tqdm(feature_counts):
        if feature_counts[ feature ] > 4 and feature_vocabulary.get(feature) is None:
            feature_vocabulary[ feature ] = feature_id
            feature_id += 1
    
    create_reverse_mappings(label_id,feature_id)
    
    max_features = feature_id
    X_train = sparse.lil_matrix((total_features,feature_id))
    y_train = list()
    
    for i in tqdm(range(total_features)):
        for feature in all_features[i]:
            if feature_vocabulary.get(feature) is not None:
                f_count = feature_vocabulary[feature]
                X_train[i,f_count] = 1
            
        y_train.append(label_vocabulary[all_labels[i]])
    
    return X_train, y_train


In [30]:
X_train, y_train = convert_to_training_data()

In [31]:
X_train.shape

(85097, 24185)

In [32]:
len(y_train)

85097

In [33]:
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import PassiveAggressiveClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB,BernoulliNB

from sklearn.model_selection import GridSearchCV
from sklearn import model_selection

## Select Model

In [34]:
def get_model(model_name):
    model = None
    if model_name == 'lr_l2':
        model = LogisticRegression(verbose = 2,n_jobs = 4)
    
    elif model_name == 'lr_el':
        model = LogisticRegression(penalty = 'l2',verbose = 2,n_jobs = 4)

    elif model_name == 'svm_rbf':
        model = SVC(kernel = 'rbf', C=1.6, probability=True)
    
    elif model_name == 'svm_poly':
        model = SVC(kernel = 'poly', C=1.5, probability=True)
    
    elif model_name == 'bnb':
        model = BernoulliNB()
        
    return model

## Fitting the data in the Model

In [35]:
import pickle

# filename = 'finalized_model.sav'
# pickle.dump(model, open(filename, 'wb'))
 
# # some time later...
 
# # load the model from disk
# loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)
# print(result)


### path to save the trained model

In [36]:
model_path = '/content/models/'

In [37]:
def train(X_train, y_train, model_name = None):
    model = get_model(model_name)
    model.fit(X_train, y_train)
    # filename = model_path + model_name + '.sav'
    # pickle.dump(model, open(filename, 'wb'))
    return model

### Training the selected model

In [38]:
trained_model = train(X_train, y_train, 'lr_l2')

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:  1.7min finished


In [39]:
def action_to_tree(test_dependency_tree,prediction_probabs,test_buffer,test_stack,test_arcs,current_transitions):

    global reverse_labels
        
    def is_action_valid(test_stack,test_buffer,action_taken):
        if action_taken == "shift" and len(test_buffer) > 0 :
            return True
        
        elif action_taken.startswith('left') and len(test_stack) > 1 and test_stack[-1] != 0: #0 is root
            return True 
        
        elif action_taken.startswith('right') and len(test_stack) > 1 and test_stack[-2] != 0: #0 is root
            return True 
        
        else:
            return False
        
        
    for label_id_prob in np.argsort(prediction_probabs, kind = 'quicksort')[0]:
        
        action_taken = reverse_labels[label_id_prob]
        
        if is_action_valid(test_stack,test_buffer,action_taken):
            current_transitions.append(action_taken)
            if action_taken == "shift":
                test_stack.append(test_buffer.pop())
            
            elif action_taken.startswith('left'):
                arc_label = action_taken.split('_')[1]
                test_dependency_tree[test_stack[-2]] = (test_stack[-1], arc_label)
                test_stack.pop(-2)
                
            elif action_taken.startswith('right'):
                arc_label = action_taken.split('_')[1]
                test_dependency_tree[test_stack[-1]] = (test_stack[-2], arc_label)
                test_stack.pop()
            return


## Evaluate and Prediction

In [40]:
def evaluate_oracles_and_predictions(test_tokens, model):
    
    global feature_vocabulary, max_features

    test_tokens_dict = defaultdict()
    test_POS_tags = defaultdict()
    
    test_tokens_dict[0] = 'root'
    test_POS_tags[0] = 'root'
    
    test_heads = defaultdict()
    test_labels = defaultdict()
    
    test_buffer = list()
    
    test_stack = list()
    test_stack.append(0)
    
    test_arcs = list()
    
    for test_token in reversed(test_tokens):
        token_id,token_word,token_tag,token_head,token_label = test_token
        
        test_tokens_dict[token_id] = token_word
        test_POS_tags[token_id] = token_tag
        
        test_heads[token_id] = token_head
        test_labels[token_id] = token_label
        
        test_buffer.append(token_id)
        
    test_dependency_tree = defaultdict()
    current_transitions = list()
    
    while len(test_buffer) > 0:
        
        if len(test_buffer) == 0  and len(test_stack)==0 : #nothing available to perform an action
            break
            
        if len(test_buffer) == 0  and len(test_stack)==1 and test_stack[0] == 0 : #nothing but root
            break
        
        current_config = (test_buffer,test_stack,test_arcs)
        
        test_features, _ = get_features_labels(current_config,current_transitions,test_tokens_dict,test_POS_tags)
        
        x_test = sparse.lil_matrix((1, max_features))
        
        for feature in test_features:
            if feature_vocabulary.get(feature) is not None:
                x_test[0, feature_vocabulary[feature]] = 1
                
        prediction_probabs = -1 * model.predict_proba(x_test)
        
        action_to_tree(test_dependency_tree,prediction_probabs,test_buffer,test_stack,test_arcs,current_transitions)
        
    unlabelled_attachment_score = 0
    labelled_attachment_score = 0
    total_children = 0
        
    for child in test_dependency_tree:
        token_head,arc_label = test_dependency_tree[child]
        if token_head == test_heads[child]:
            unlabelled_attachment_score += 1
            if arc_label == test_labels[child]:
                labelled_attachment_score += 1
        total_children += 1
            
    return [unlabelled_attachment_score,labelled_attachment_score,total_children]


In [41]:
gg = 0
def evaluate(test_treebank, model):
    global gg
    test_tokens = [] #tokens in current line
    current_all_scores = []
    total_scores = np.zeros(3)
    
    new_idx = test_treebank[0][0]
    for current_line in tqdm(test_treebank):
        if current_line[0] == new_idx:
            tok_id,word,x_pos,head_id,dep_rel = current_line[0], current_line[1], current_line[2], current_line[-1], current_line[6] 
            test_tokens.append((tok_id,word,x_pos,head_id,dep_rel))
        else:
            new_test_tokens = []
            cnt_root = 0
            for idx , line in enumerate(test_tokens):
                temp = list(line)
                temp[0] = idx + 1
                new_test_tokens.append(tuple(temp))
                if temp[-1] == 'root' :
                    cnt_root += 1
            
            
            if cnt_root == 1 and check_projectivity(new_test_tokens) == True: #if current tree is projective
                gg += 1
                current_all_scores = evaluate_oracles_and_predictions(new_test_tokens,model)
                total_scores += current_all_scores

            test_tokens = []
            new_idx = current_line[0]
            tok_id,word,x_pos,head_id,dep_rel = current_line[0], current_line[1], current_line[2], current_line[-1], current_line[6] 
            test_tokens.append((tok_id,word,x_pos,head_id,dep_rel))

    new_test_tokens = []
    cnt_root = 0
    for idx , line in enumerate(test_tokens):
        temp = list(line)
        temp[0] = idx + 1
        new_test_tokens.append(tuple(temp))
        if temp[-1] == 'root' :
            cnt_root += 1
    if cnt_root == 1 and check_projectivity(new_test_tokens) == True: #if current tree is projective
        gg += 1
        current_all_scores = evaluate_oracles_and_predictions(new_test_tokens,model)
        total_scores += current_all_scores
        

    accuracy = f'Unlabelled attachment Score {total_scores[0]/total_scores[2]} Labelled attachment Score {total_scores[1]/total_scores[2]}'
    
    return accuracy, total_scores

In [42]:
accuracy, total_scores = evaluate(test_treebank, trained_model)

In [43]:
accuracy

'Unlabelled attachment Score 0.8421052631578947 Labelled attachment Score 0.631578947368421'